In [1]:
import pandas as pd

In [2]:
tr=pd.read_csv('C:/Users/cmscomputers/Downloads/Skype/Datasets/Google_Stock_Price_Train.csv')

In [3]:
ts =pd.read_csv('C:/Users/cmscomputers/Downloads/Skype/Datasets/Google_Stock_Price_Test.csv')

# Creation of data

In [4]:
data=pd.concat([tr,ts])

In [5]:
data.index=range(0,1278)

In [6]:
data

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"
...,...,...,...,...,...,...
1273,1/25/2017,829.62,835.77,825.06,835.67,"1,494,500"
1274,1/26/2017,837.81,838.00,827.01,832.15,"2,973,900"
1275,1/27/2017,834.71,841.95,820.44,823.31,"2,965,800"
1276,1/30/2017,814.66,815.84,799.80,802.32,"3,246,600"


In [7]:
data=data[['Open']]

In [8]:
train=data.iloc[:1178]
test=data.iloc[1178:]

# Standerdisation

In [9]:
from sklearn.preprocessing import MinMaxScaler
mm=MinMaxScaler()
mm.fit(train)
scaled_train=mm.fit_transform(train)
scaled_test=mm.fit_transform(test)

# Preparing data Sequence for RNN

In [10]:
xtrain=[]
for i in range(50,1178,1):
    xtrain.append(scaled_train[i-50:i])

In [11]:
ytrain=[]
for i in range(50,1178,1):
    ytrain.append(scaled_train[i:i+1])

In [12]:
import numpy as np
xtrain1=np.array(xtrain)
ytrain1=np.array(ytrain)

In [13]:
xtrain1.shape

(1128, 50, 1)

In [14]:
ytrain1.shape

(1128, 1, 1)

In [15]:
scaled_test.shape

(100, 1)

In [16]:
xtest=[]
ytest=[]
for i in range(50,100,1):
    xtest.append(scaled_test[i-50:i])
    ytest.append(scaled_test[i:i+1])

In [17]:
xtest1=np.array(xtest)
ytest1=np.array(ytest)

In [18]:
xtest1.shape

(50, 50, 1)

In [19]:
ytest1.shape

(50, 1, 1)

# Creating a RNN 

In [20]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout

In [21]:
nn=Sequential()

nn.add(LSTM(100,return_sequences=True,input_shape=(50,1)))                     #input layer

nn.add(LSTM(100,return_sequences=True))                                         #Hidden layer

nn.add(Dense(1))

In [239]:
nn.compile(optimizer='adam',loss='mean_absolute_error')
model=nn.fit(xtrain1,ytrain1,epochs=5000,initial_epoch=5000,batch_size=128)

# Predictions

In [204]:
pred_tr=nn.predict(xtrain1)
pred_ts=nn.predict(xtest1)

In [205]:
print(pred_tr.shape)
print(pred_ts.shape)

(1128, 50, 1)
(50, 50, 1)


In [206]:
ytr=ytrain1.reshape((1128,1))
yts=ytest1.reshape(50,1)

In [207]:
pr=pred_tr[:,0].reshape((1128,1))
ps=pred_ts[0].reshape((50,1))          #selectig a chunk for Error prediction

# Destanderdisation

In [208]:
tr=mm.inverse_transform(pr)
ts=mm.inverse_transform(ps)              #predicted

In [209]:
ytr=mm.inverse_transform(ytr)
yts=mm.inverse_transform(yts)                 #target feature

# Error calculation

In [210]:
from sklearn.metrics import mean_absolute_error
err_tr=mean_absolute_error(tr,ytr)
err_ts=mean_absolute_error(ts,yts)

In [211]:
err_tr

5.278690002008975

In [212]:
err_ts

13.813420019531245